# Fisher Score- Chisquare Test For Feature Selection

Compute chi-squared stats between each non-negative feature and class.

This score should be used to evaluate categorical variables in a classification task.

This score can be used to select the n_features features with the highest values for the test chi-squared statistic from X, which must contain only non-negative features such as booleans or frequencies (e.g., term counts in document classification), relative to the classes.

Recall that the chi-square test measures dependence between stochastic variables, so using this function “weeds out” the features that are the most likely to be independent of class and therefore irrelevant for classification. The Chi Square statistic is commonly used for testing relationships between categorical variables.

It compares the observed distribution of the different classes of target Y among the different categories of the feature, against the expected distribution of the target classes, regardless of the feature categories

# Import Inbuild Titanic Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
# Load data
data = sns.load_dataset('titanic')
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [4]:
# Make a dataframe of categorical feature
df = data[['sex','pclass','embarked','class','alone','survived']]
df.head()

,sex,pclass,embarked,class,alone,survived
0,male,3,S,Third,False,0
1,female,1,C,First,False,1
2,female,3,S,Third,True,1
3,female,1,S,First,False,1
4,male,3,S,Third,True,0


In [5]:
# Convert Categorical to numerical using labels encoding sex feature
df.sex = np.where(df['sex']=='male',1,0)

C:\Users\hp\AppData\Local\Temp\ipykernel_12492\982177211.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sex = np.where(df['sex']=='male',1,0)


In [6]:
df.sex.unique()

array([1, 0])

In [9]:
# Convert embarked feature into numerical using label encoding
ordinal_label = {k: i for i, k  in enumerate(df['embarked'].unique(),0)}
df['embarked'] = df['embarked'].map(ordinal_label)

C:\Users\hp\AppData\Local\Temp\ipykernel_12492\3655045739.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embarked'] = df['embarked'].map(ordinal_label)


In [10]:
df.embarked.unique()

array([0, 1, 2, 3], dtype=int64)

In [13]:
df['class'].value_counts()

Third     491
First     216
Second    184
Name: class, dtype: int64

In [18]:
# Convert class into numerical
df['class'] = df['class'].map({"Third":2,'First':1,'Second':0})

C:\Users\hp\AppData\Local\Temp\ipykernel_12492\437625749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['class'] = df['class'].map({"Third":2,'First':1,'Second':0})


In [19]:
df.head()

,sex,pclass,embarked,class,alone,survived
0,1,3,0,2,False,0
1,0,1,1,1,False,1
2,0,3,0,2,True,1
3,0,1,0,1,False,1
4,1,3,0,2,True,0


In [20]:
df['alone'] = np.where(df['alone']==True,1,0)

C:\Users\hp\AppData\Local\Temp\ipykernel_12492\1503327378.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['alone'] = np.where(df['alone']==True,1,0)


In [21]:
df.head()

,sex,pclass,embarked,class,alone,survived
0,1,3,0,2,0,0
1,0,1,1,1,0,1
2,0,3,0,2,1,1
3,0,1,0,1,0,1
4,1,3,0,2,1,0


### Split Training and testing data

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['sex','pclass','embarked','class','alone']],
                                                   df['survived'],
                                                   test_size=0.30,
                                                   random_state=45)

In [27]:
X_train.head()

,sex,pclass,embarked,class,alone
650,1,3,0,2,1
784,1,3,0,2,1
125,1,3,1,2,0
54,1,1,1,1,0
446,0,2,0,0,0


## Chi-Square [Fisher Score] 

In [28]:
from sklearn.feature_selection import chi2

f_p_value = chi2(X_train,y_train)
f_p_value

(array([59.51852849, 20.4485877 , 10.87947637, 14.76568257,  9.04673587]),
 array([1.21148117e-14, 6.12547407e-06, 9.72357332e-04, 1.21730827e-04,
        2.63164242e-03]))

In [32]:
# Convert p vlaues into series
p_value = pd.Series(f_p_value[1])
p_value.index = X_train.columns
p_value

sex         1.211481e-14
pclass      6.125474e-06
embarked    9.723573e-04
class       1.217308e-04
alone       2.631642e-03
dtype: float64

In [33]:
p_value.sort_index(ascending=False)

sex         1.211481e-14
pclass      6.125474e-06
embarked    9.723573e-04
class       1.217308e-04
alone       2.631642e-03
dtype: float64

## Observation
Sex Column is the most important column when compared to the output feature Survived